In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import h5py
import logging
from os import path
from pandas import HDFStore
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import pathlib
import shutil
import tempfile
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots


%matplotlib inline

In [12]:
#!pip install -q git+https://github.com/tensorflow/docs

In [33]:
logdir = pathlib.Path("C:/tmp/")/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)

In [34]:
logdir

WindowsPath('C:/tmp/tensorboard_logs')

In [35]:
#Load data from hdf5 file
root_path = "Z:/elekin"
rdo_root_path = path.join(root_path,"02-RESULTADOS/03-HANDWRITTING")
h5file = path.join(rdo_root_path, "00-OUTPUT/archimedean-")
h5filename = h5file + str(17) + ".h5"
hdf = HDFStore(h5filename)

In [36]:
#Load and scale timeseries between 0 and 1
raw_df = hdf['results/residues/rd'].T
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_df = pd.DataFrame(scaler.fit_transform(raw_df))
scaled_df['target'] = hdf.get('source/labels').values

In [37]:
# split data into train and test datasets (0.8, 0.2)
shuffle_df = scaled_df.sample(frac=1).reset_index(drop=True)
msk = np.random.rand(len(shuffle_df)) < 0.8
x_train = shuffle_df[msk].iloc[:,0:4096].values.astype('float32')
y_train = shuffle_df[msk]['target'].values.astype('int8')
x_test = shuffle_df[~msk].iloc[:,0:4096].values.astype('float32')
y_test = shuffle_df[~msk]['target'].values.astype('int8')

In [38]:
#Early stop configuration
earlystop_callback = EarlyStopping(
  monitor='val_accuracy', min_delta=0.0001,
  patience=20)

training_earlystop_callback = EarlyStopping(
  monitor='accuracy', min_delta=0.0001,
  patience=5)

def get_callbacks(name):
    return [
        tfdocs.modeling.EpochDots(),
        tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
        tf.keras.callbacks.TensorBoard(logdir/name),
      ]

def compile_and_fit(model, name, optimizer=None, max_epochs=1000):
    if optimizer is None:
        optimizer = get_optimizer()
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'binary_crossentropy'])
        model.summary()
        history = model.fit(x_train, y_train, batch_size=None, use_multiprocessing=True, 
                            validation_data=(x_test,y_test), epochs=max_epochs, callbacks=get_callbacks(name),
                            verbose=0)
        return history

In [39]:
tf.random.set_seed(2019) # establecemos la semilla para tensorflow
np.random.seed(2019) # establecemos la semilla para numpy

## CNN

Many models train better if you gradually reduce the learning rate during training. 
Use optimizers.schedules to reduce the learning rate over time:

In [40]:
STEPS_PER_EPOCH = 1
FEATURES = 4096

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

In [ ]:
tf.keras.backend.clear_session()# para evitar que entrenamientos annteriores afecten
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),# expand the dimension form (50, 4096) to (50, 4096, 1)
                      input_shape=[4096]),
    tf.keras.layers.Conv1D(filters=16, kernel_size=5,
                       activation=tf.nn.relu,
                      input_shape=[4096, 1]),
    tf.keras.layers.MaxPool1D(),
    tf.keras.layers.Conv1D(filters=16, kernel_size=5,
                           activation=tf.nn.relu),
#    tf.keras.layers.LSTM(128, activation=tf.nn.relu),
     tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
     tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])

In [42]:
size_histories = {}

In [ ]:
size_histories['cnn/conv1d'] = compile_and_fit(tiny, "cnn/conv1d", optimizer=None, max_epochs=1000)

In [ ]:
plotter_loss = tfdocs.plots.HistoryPlotter(metric = 'binary_crossentropy', smoothing_std=10)
plotter_loss.plot(size_histories)
plt.ylim([0., 1.])

In [ ]:
plotter_acc = tfdocs.plots.HistoryPlotter(metric = 'accuracy', smoothing_std=10)
plotter_acc.plot(size_histories)
plt.ylim([0.3, 0.9])

In [ ]:
plotter_acc.plot(size_histories)
a = plt.xscale('log')
plt.xlim([5, max(plt.xlim())])
plt.ylim([0.5, 0.9])
plt.xlabel("Epochs [Log Scale]")

In [ ]:
plotter_loss.plot(size_histories)
a = plt.xscale('log')
plt.xlim([5, max(plt.xlim())])
plt.ylim([0., 0.9])
plt.xlabel("Epochs [Log Scale]")

### Tensorboard

In [26]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard
%tensorboard --logdir {logdir}/sizes

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 244.